In [217]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder,OneHotEncoder,RobustScaler
from sklearn.metrics import accuracy_score , mean_absolute_error,mean_squared_error , r2_score 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
df=pd.read_csv('CreditCardBalance.csv')
df.head()

,Unnamed: 0,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [4]:
data_cleaned = df.drop(columns=["Unnamed: 0"])

In [5]:
X = data_cleaned.drop(columns=["Balance"])
y = data_cleaned["Balance"]

In [15]:
data_cleaned["Rating"].median()


344.0

In [221]:
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(include=["number"]).columns

In [222]:
numerical_transformer = StandardScaler()

In [223]:
categorical_transformer = OneHotEncoder(drop="first", handle_unknown="ignore")


In [224]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols),
    ]
)

In [225]:
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

In [226]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [227]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [228]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R²):", r2)

Mean Absolute Error (MAE): 74.32414917369795
Mean Squared Error (MSE): 7974.85644954117
R-squared (R²): 0.9522674050276403


In [229]:
import nbformat
import os

def convert_ipynb_to_py(ipynb_filename):
    if not ipynb_filename.endswith(".ipynb"):
        print("Error: Please provide a valid .ipynb file.")
        return
    
    py_filename = os.path.splitext(ipynb_filename)[0] + ".py"
    
    try:
        with open(ipynb_filename, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
        
        with open(py_filename, 'w', encoding='utf-8') as f:
            for cell in notebook.cells:
                if cell.cell_type == "code":
                    f.write("\n".join(cell.source.splitlines()) + "\n\n")
        
        print(f"Successfully converted {ipynb_filename} to {py_filename}")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    filename = input("Enter the Jupyter Notebook filename (with .ipynb extension): ")
    convert_ipynb_to_py(filename)


Error: Please provide a valid .ipynb file.


In [230]:
import pickle

model_data = {
    "model": model,
    "features": X.columns.tolist()
}

with open("credit_model.pkl", "wb") as f:
    pickle.dump(model_data, f)

print("Model saved as credit_model.pkl")

Model saved as credit_model.pkl


In [ ]:
import logging
import pickle
import pandas as pd
import numpy as np
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the trained pipeline
try:
    with open("credit_model.pkl", "rb") as f:
        model_schema = pickle.load(f)
    
    model = model_schema.get("model")
    features = model_schema.get("features", [])

    if model is None or not features:
        raise ValueError("Model or features missing in pickle file.")

    logger.info("Model loaded successfully.")

except Exception as e:
    logger.error(f"Error loading model: {e}")
    model = None
    features = []

# FastAPI App
app = FastAPI()

# Allow CORS for all origins or specific React origin
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins. You can restrict this to specific origins (e.g., ["http://localhost:3000"] for React dev server)
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods (GET, POST, etc.)
    allow_headers=["*"],  # Allow all headers
)

# Define request schema
class CreditApplication(BaseModel):
    Income: float
    Limit: int
    Rating: int
    Cards: int
    Age: int
    Education: int
    Gender: str
    Student: str
    Married: str
    Ethnicity: str

# Mapping education levels to a multiplier (1 being lowest, 1.5 being highest)
education_multiplier = {
    5: 1.0,   # Social Work
    6: 1.0,   # Psychology
    7: 1.0,   # Agricultural Engineering
    8: 1.1,   # Journalism and Media
    9: 1.1,   # Applied Arts
    10: 1.1,  # Education
    11: 1.1,  # Public Health
    12: 1.2,  # Political Science
    13: 1.2,  # Financial Engineering
    14: 1.2,  # Nursing
    15: 1.2,  # Veterinary Science
    16: 1.3,  # Architecture
    17: 1.3,  # Economics
    18: 1.3,  # Business Administration
    19: 1.4,  # Law
    20: 1.5   # Computer Science, Engineering, Medicine, Dentistry, Pharmacy
}

def preprocess_input(data: CreditApplication):
    df = pd.DataFrame([data.dict()])

    missing_cols = [col for col in features if col not in df.columns]
    for col in missing_cols:
        df[col] = 0  

    # Reorder columns to match training set
    df = df.reindex(columns=features, fill_value=0)

    return df

# Prediction endpoint
@app.post("/predict")
async def predict_credit(data: CreditApplication):
    if model is None:
        return {"error": "Model is not loaded properly."}

    try:
        processed_data = preprocess_input(data)
        prediction = model.predict(processed_data)

        predicted_balance = float(np.expm1(prediction[0])) if "log" in model_schema else float(prediction[0])

        # Fetching the education multiplier based on the education level
        education_factor = education_multiplier.get(data.Education, 1.0)  # Default to 1 if not found

        # Improved Loan Calculation with education factor
        loan_amount = min(
            data.Income * 5,  # Max 5x Annual Income
            (data.Limit * 0.3) +
            (data.Rating * 8) - 
            (data.Cards * 300) - 
            (predicted_balance * 0.4)  # Deduct 40% of predicted balance
        )

        loan_amount *= education_factor

        loan_amount = max(0, loan_amount)

        return {"Loan amount you can take": loan_amount}
    except Exception as e:
        logger.error(f"Prediction error: {e}")
        return {"error": str(e)}

# Run FastAPI
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8002)


INFO:__main__:Model loaded successfully.
INFO:     Started server process [16332]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8002 (Press CTRL+C to quit)


INFO:     127.0.0.1:53338 - "OPTIONS /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:53338 - "POST /predict HTTP/1.1" 200 OK


c:\Users\dell\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\dell\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


INFO:     127.0.0.1:53338 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:53352 - "POST /predict HTTP/1.1" 200 OK


c:\Users\dell\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


...

c:\Users\dell\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


INFO:     127.0.0.1:53434 - "POST /predict HTTP/1.1" 200 OK


c:\Users\dell\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


INFO:     127.0.0.1:53434 - "POST /predict HTTP/1.1" 200 OK


c:\Users\dell\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
